In [171]:
import cv2
import numpy as np
import pandas as pd
import os

In [172]:
summary_df = pd.DataFrame(columns=["id", "trial", "left_eye", "right_eye", "nose", "mouth", "outside"])
os.makedirs("exported_csv/fixation_counts", exist_ok=True)

In [173]:
subject_id = 1
experiment_id = 1
trial_num = 0
# 顔画像の読み込み
img = cv2.imread(f'../face_aoi_project/output_aoi/{experiment_id}-{trial_num + 1}.jpg')
fixation_df = pd.read_csv(f"exported_csv/fixation_IDT/fix_df_{subject_id:03}-{experiment_id:03}-{trial_num}.csv")
total_fix = len(fixation_df)

In [174]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lower_cyan = np.array([80, 50, 50])
upper_cyan = np.array([100, 255, 255])
mask_blue = cv2.inRange(hsv, lower_cyan, upper_cyan)
kernel = np.ones((3, 3), np.uint8)
mask_clean = cv2.morphologyEx(mask_blue, cv2.MORPH_CLOSE, kernel)

In [175]:
# 輪郭抽出
contours, _ = cv2.findContours(mask_blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# 面積の大きい輪郭から上位4つをAOIとして採用
sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)[:4]

In [176]:
aoi_masks = []
aoi_counts = []

for i, contour in enumerate(sorted_contours):
    # AOIごとのマスク作成
    mask = np.zeros(mask_clean.shape, dtype=np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)
    aoi_masks.append(mask)

    # 注視点カウント（ピクセル内包判定）
    count = 0
    for _, row in fixation_df.iterrows():
        x, y = int(row['x_px']), int(row['y_px'])
        if 0 <= x < mask.shape[1] and 0 <= y < mask.shape[0]:
            if mask[y, x] == 255:
                count += 1
    aoi_counts.append(count)

In [177]:
# AOI情報まとめ（重心・ラベル・カウント）
aoi_info = []

overlay = img.copy()

# AOIマスクを赤で半透明に重ねる
for mask in aoi_masks:
    red_mask = np.zeros_like(img)
    red_mask[mask == 255] = [0, 0, 255]
    overlay = cv2.addWeighted(overlay, 1.0, red_mask, 0.4, 0)

# 注視点を緑で描く
for _, row in fixation_df.iterrows():
    cv2.circle(overlay, (int(row['x_px']), int(row['y_px'])), 4, (0, 255, 0), -1)

for contour, count in zip(sorted_contours, aoi_counts):
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        aoi_info.append({'contour': contour, 'count': count, 'cx': cx, 'cy': cy})


In [178]:
# === 並び替えとラベル付け ===
aoi_info_sorted = sorted(aoi_info, key=lambda d: d['cy'])
eyes = sorted(aoi_info_sorted[:2], key=lambda d: d['cx'])
nose = aoi_info_sorted[2]
mouth = aoi_info_sorted[3]

# === ラベル付きAOI ===
aoi_labeled = [
    {'label': 'left_eye',  'count': eyes[0]['count'], 'cx': eyes[0]['cx'], 'cy': eyes[0]['cy']},
    {'label': 'right_eye', 'count': eyes[1]['count'], 'cx': eyes[1]['cx'], 'cy': eyes[1]['cy']},
    {'label': 'nose',      'count': nose['count'],    'cx': nose['cx'],    'cy': nose['cy']},
    {'label': 'mouth',     'count': mouth['count'],   'cx': mouth['cx'],   'cy': mouth['cy']}
]

In [179]:
# === AOI外の注視数を計算 ===
inside_total = sum([aoi['count'] for aoi in aoi_labeled])
outside_count = total_fix - inside_total

# === outside の行を追加 ===
aoi_labeled.append({
    'label': 'outside',
    'count': outside_count,
    'cx': np.nan,
    'cy': np.nan
})

In [181]:
# === DataFrame化 ===
df_result = pd.DataFrame(aoi_labeled)
df_result.insert(1, 'total_fixation', total_fix)

# === 出力確認 ===
print(df_result)

       label  total_fixation  count      cx     cy
0   left_eye              86      4   849.0  546.0
1  right_eye              86      3  1097.0  559.0
2       nose              86      1   969.0  676.0
3      mouth              86      2   965.0  852.0
4    outside              86     76     NaN    NaN


In [180]:
# # === result_df 保存 ===
# summary_path = "exported_csv/fixation_counts/summary_all_trials.csv"
# summary_df.to_csv(summary_path, index=False, encoding='utf-8-sig')

In [182]:
# === summary_df に1行追加 ===
summary_row = {
    "id": f"{subject_id:03}-{experiment_id:03}",
    "trial": trial_num
}
for label in ["left_eye", "right_eye", "nose", "mouth", "outside"]:
    count = df_result.loc[df_result["label"] == label, "count"].values
    summary_row[label] = int(count[0]) if len(count) > 0 else 0

summary_df = pd.concat([summary_df, pd.DataFrame([summary_row])], ignore_index=True)
print(summary_df)

        id trial left_eye right_eye nose mouth outside
0  001-001     0        4         3    1     2      76


In [183]:

# # === summary_df 保存 ===
# summary_path = "exported_csv/fixation_counts/summary_all_trials.csv"
# summary_df.to_csv(summary_path, index=False, encoding='utf-8-sig')
# print(f"summary_df 保存済: {summary_path}")

In [184]:
scale = 0.5
small_overlay = cv2.resize(overlay, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
cv2.imshow("AOI Overlay with Fixations", small_overlay)
cv2.waitKey(0)
cv2.destroyAllWindows()